In [1]:
class EnviromentReport():
    
    def __init__(self):
        self.energy_consumption_entries = []
        
    def add_entry(self, amount_energy, total_time, vessel):
        self.energy_consumption_entries.append((amount_energy, total_time, vessel))
        
    def get_energy_usage(self):
        filtered_result = {}
        for energy_entry in self.energy_consumption_entries:
            vessel = energy_entry[2]
            path_name = vessel.path_complete.name
            
            if path_name in filtered_result:
                if (vessel.loaded):
                    filtered_result[path_name]["loaded"] += energy_entry[0]
                else:
                    filtered_result[path_name]["unloaded"] += energy_entry[0]
            else:
                if (vessel.loaded):
                    filtered_result[path_name] = {"loaded": energy_entry[0], "unloaded": 0}
                else:
                    filtered_result[path_name] = {"loaded": 0, "unloaded": energy_entry[0]}

        return filtered_result
    
    def get_co2_emission_load_km(self):
        filtered_result = {}
        moved_load_km = {}
        for energy_entry in self.energy_consumption_entries:
            vessel = energy_entry[2]
            path_name = vessel.path_complete.name
            
            if path_name in moved_load_km and vessel.loaded:
                moved_load_km[path_name] += vessel.distance * vessel.loaded_capacity

            if (path_name not in moved_load_km and vessel.loaded):
                moved_load_km[path_name] = vessel.distance * vessel.loaded_capacity
            
            if path_name in filtered_result:
                    filtered_result[path_name] += energy_entry[0] * 0.2 * vessel.emission *10**3 

            else:
                    filtered_result[path_name] = energy_entry[0] * 0.2 * vessel.emission * 10**3
        
        for key in filtered_result.keys():
            filtered_result[key] = filtered_result[key] / (10**-3 * moved_load_km[key])
        return filtered_result
    
    
    def get_co2_emission(self):
        filtered_result = {}
        for energy_entry in self.energy_consumption_entries:
            vessel = energy_entry[2]
            path_name = vessel.path_complete.name
            
            if path_name in filtered_result:
                if (vessel.loaded):
                    filtered_result[path_name]["loaded"] += energy_entry[0] * 0.2 * vessel.emission
                else:
                    filtered_result[path_name]["unloaded"] += energy_entry[0] * 0.2 * vessel.emission
            else:
                if (vessel.loaded):
                    filtered_result[path_name] = {"loaded": energy_entry[0] * 0.2 * vessel.emission, "unloaded": 0}
                else:
                    filtered_result[path_name] = {"loaded": 0, "unloaded": energy_entry[0] * 0.2 * vessel.emission}

        return filtered_result
    
    def plot_co2_emission(self, save=False):
        plot_dict = self.get_co2_emission()
        
        loaded = [i["loaded"] for i in plot_dict.values()]
        unloaded = [i["unloaded"] for i in plot_dict.values()]
        loaded.append(np.sum(loaded))
        unloaded.append(np.sum(unloaded))
        plot_dict["total"] = {}
        width = 0.35       # the width of the bars: can also be len(x) sequence
        plt.figure(figsize=(6,6))
        plt_unloaded = plt.bar(range(len(plot_dict)), unloaded, width, bottom=loaded, align='center')
        plt_loaded = plt.bar(range(len(plot_dict)), loaded, width, align='center')
        
        plt.xticks(range(len(plot_dict)), list(plot_dict.keys()))
        plt.legend((plt_unloaded[0], plt_loaded[0]), ('unloaded vessels', 'Loaded vessels'))
        
        plt.ylabel('Kg')
        plt.title('Total fleet CO2 emission over {} hours catagorized by paths'\
                  .format(int(self.energy_consumption_entries[-1][2].env.now / 3600)))
        #plt.title('{} vessels'.format(len(self.energy_consumption_entries)))
        if save:
            plt.savefig()
        plt.show()
        
    def plot_co2_emission_km_load(self, save=False):
        plot_dict =  self.get_co2_emission_load_km()
        #plot_dict["total"] = np.sum(list(plot_dict.values()))
        width = 0.35       # the width of the bars: can also be len(x) sequence
        plt.figure(figsize=(5,5))
        
        plt.bar(range(len(plot_dict)), list(plot_dict.values()), width, align='center')
        
        plt.xticks(range(len(plot_dict)), list(plot_dict.keys()))

        
        plt.ylabel('g/Ton KM')
        plt.title('Total fleet CO2/Ton Km emission over {} hours catagorized by paths'\
                  .format(int(self.energy_consumption_entries[-1][2].env.now / 3600)))
        #plt.title('{} vessels'.format(len(self.energy_consumption_entries)))
        if save:
            plt.savefig()
        plt.show()
        
    
    def plot_energy_usage(self, save=False):
        plot_dict = self.get_energy_usage()
        
        loaded = [i["loaded"] for i in plot_dict.values()]
        unloaded = [i["unloaded"] for i in plot_dict.values()]
        loaded.append(np.sum(loaded))
        unloaded.append(np.sum(unloaded))
        plot_dict["total"] = {}
        width = 0.35       # the width of the bars: can also be len(x) sequence
        plt.figure(figsize=(6,6))
        plt_unloaded = plt.bar(range(len(plot_dict)), unloaded, width, bottom=loaded, align='center')
        plt_loaded = plt.bar(range(len(plot_dict)), loaded, width, align='center')
        
        plt.xticks(range(len(plot_dict)), list(plot_dict.keys()))
        plt.legend((plt_unloaded[0], plt_loaded[0]), ('unloaded vessels', 'Loaded vessels'))
        
        plt.ylabel('KWh')
        plt.title('Total fleet energy usage over {} hours catagorized by paths'\
                  .format(int(self.energy_consumption_entries[-1][2].env.now / 3600)))
        #plt.title('{} vessels'.format(len(self.energy_consumption_entries)))
        if save:
            plt.savefig()
        plt.show()
        
        
        
    def plot_path_usage(self):
        plt.figure(figsize=(6,6))
        plt.title("Vessel distribution over paths")
        plt.ylabel('Number of vessels')
        plot_dict = self.energy_consumption_entries[-1][2].env.paths


        plot_dict["total"] = np.sum(list(plot_dict.values()))
        plt.bar(range(len(plot_dict)), list(plot_dict.values()), align='center')
        plt.xticks(range(len(plot_dict)), list(plot_dict.keys()))

    def report(self):
        print(self.energy_consumption_entries)
        